In [101]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [102]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np

In [103]:
# fetch dataset 
metro_interstate_traffic_volume = fetch_ucirepo(id=492) 
  
# data (as pandas dataframes) 
X = metro_interstate_traffic_volume.data.features 
y = metro_interstate_traffic_volume.data.targets   

In [104]:
X.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00


In [105]:
X

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00
...,...,...,...,...,...,...,...,...
48199,NaN,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00
48200,NaN,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00


In [106]:
X.iloc[0,7]

'2012-10-02 09:00:00'

In [107]:
X.iloc[-1,7]

'2018-09-30 23:00:00'

In [108]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              61 non-null     object 
 1   temp                 48204 non-null  float64
 2   rain_1h              48204 non-null  float64
 3   snow_1h              48204 non-null  float64
 4   clouds_all           48204 non-null  int64  
 5   weather_main         48204 non-null  object 
 6   weather_description  48204 non-null  object 
 7   date_time            48204 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 2.9+ MB


In [109]:
y.head()

,traffic_volume
0,5545
1,4516
2,4767
3,5026
4,4918


In [110]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   traffic_volume  48204 non-null  int64
dtypes: int64(1)
memory usage: 376.7 KB


biến đổi holiday

In [111]:
X['holiday'].unique()

array([nan, 'Columbus Day', 'Veterans Day', 'Thanksgiving Day',
       'Christmas Day', 'New Years Day', 'Washingtons Birthday',
       'Memorial Day', 'Independence Day', 'State Fair', 'Labor Day',
       'Martin Luther King Jr Day'], dtype=object)

In [112]:
# 1. Tạo cột is_holiday (0/1)
X["is_holiday"] = X["holiday"].notna().astype(int)

# 2. Xóa cột holiday gốc
X = X.drop(columns=["holiday"])

biến đổi datetime

In [113]:
# --- 1. Convert datetime ---
date_time_copy = pd.to_datetime(X["date_time"])


In [114]:
start = date_time_copy.min()
end = date_time_copy.max()
print(start, end)


2012-10-02 09:00:00 2018-09-30 23:00:00


In [115]:
start = date_time_copy.min()
end = date_time_copy.max()
print(start, end)

2012-10-02 09:00:00 2018-09-30 23:00:00


In [116]:
total_hours = int((end - start) / pd.Timedelta(hours=1)) + 1
print("Số giờ liên tục dự kiến:", total_hours)


Số giờ liên tục dự kiến: 52551


In [117]:
print("Số bản ghi trong dataset:", len(X))

Số bản ghi trong dataset: 48204


In [118]:
train = X[X["date_time"] < "2018-01-01"]
test  = X[X["date_time"] >= "2018-01-01"]

In [119]:
# --- 0. Convert datetime ---
X["date_time"] = pd.to_datetime(X["date_time"])

# --- 1. Extract base time features ---
X["year"] = X["date_time"].dt.year
X["month"] = X["date_time"].dt.month
X["day"] = X["date_time"].dt.day
X["hour"] = X["date_time"].dt.hour
X["day_of_week"] = X["date_time"].dt.dayofweek

# --- 2. Cyclical encoding ---
# Hour
X["hour_sin"] = np.sin(2 * np.pi * X["hour"] / 24)
X["hour_cos"] = np.cos(2 * np.pi * X["hour"] / 24)
# Day of week
X["dow_sin"] = np.sin(2 * np.pi * X["day_of_week"] / 7)
X["dow_cos"] = np.cos(2 * np.pi * X["day_of_week"] / 7)
# Month
X["month_sin"] = np.sin(2 * np.pi * X["month"] / 12)
X["month_cos"] = np.cos(2 * np.pi * X["month"] / 12)

In [120]:
# --- 3. Chia train/test theo thời gian ---
train_idx = X["date_time"] < "2018-01-01"
test_idx  = X["date_time"] >= "2018-01-01"

train_X = X.loc[train_idx].copy()
test_X  = X.loc[test_idx].copy()

train_y = y.loc[train_idx].copy()
test_y  = y.loc[test_idx].copy()

In [121]:
# --- 4. Tạo lag/rolling features trên toàn bộ X (train+test) để đảm bảo lag từ train cho test ---
combined_X = pd.concat([train_X, test_X])
combined_y = pd.concat([train_y, test_y])

# Lag features
combined_X["traffic_volume_lag1"]   = combined_y.shift(1)
combined_X["traffic_volume_lag24"]  = combined_y.shift(24)
combined_X["traffic_volume_lag168"] = combined_y.shift(168)

# Rolling features
combined_X["traffic_roll_mean_3"]  = combined_y.shift(1).rolling(window=3).mean()
combined_X["traffic_roll_mean_24"] = combined_y.shift(1).rolling(window=24).mean()
combined_X["traffic_roll_std_24"]  = combined_y.shift(1).rolling(window=24).std()

# Drop các hàng NaN do lag/rolling
combined_X = combined_X.dropna().reset_index(drop=True)
combined_y = combined_y.loc[combined_X.index]  # đồng bộ target

# Xóa cột date_time
combined_X.drop('date_time', axis=1, inplace=True)

# --- 5. Tách lại train/test sau khi lag/rolling ---
X_train = combined_X.loc[train_idx[combined_X.index]].copy()
X_test  = combined_X.loc[test_idx[combined_X.index]].copy()
y_train = combined_y.loc[train_idx[combined_X.index]].copy()
y_test  = combined_y.loc[test_idx[combined_X.index]].copy()

In [122]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40255 entries, 0 to 40254
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   temp                   40255 non-null  float64
 1   rain_1h                40255 non-null  float64
 2   snow_1h                40255 non-null  float64
 3   clouds_all             40255 non-null  int64  
 4   weather_main           40255 non-null  object 
 5   weather_description    40255 non-null  object 
 6   is_holiday             40255 non-null  int64  
 7   year                   40255 non-null  int32  
 8   month                  40255 non-null  int32  
 9   day                    40255 non-null  int32  
 10  hour                   40255 non-null  int32  
 11  day_of_week            40255 non-null  int32  
 12  hour_sin               40255 non-null  float64
 13  hour_cos               40255 non-null  float64
 14  dow_sin                40255 non-null  float64
 15  dow_cos

In [123]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7781 entries, 40255 to 48035
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   temp                   7781 non-null   float64
 1   rain_1h                7781 non-null   float64
 2   snow_1h                7781 non-null   float64
 3   clouds_all             7781 non-null   int64  
 4   weather_main           7781 non-null   object 
 5   weather_description    7781 non-null   object 
 6   is_holiday             7781 non-null   int64  
 7   year                   7781 non-null   int32  
 8   month                  7781 non-null   int32  
 9   day                    7781 non-null   int32  
 10  hour                   7781 non-null   int32  
 11  day_of_week            7781 non-null   int32  
 12  hour_sin               7781 non-null   float64
 13  hour_cos               7781 non-null   float64
 14  dow_sin                7781 non-null   float64
 15  dow_

In [124]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40255 entries, 0 to 40254
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   traffic_volume  40255 non-null  int64
dtypes: int64(1)
memory usage: 629.0 KB


In [125]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7781 entries, 40255 to 48035
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   traffic_volume  7781 non-null   int64
dtypes: int64(1)
memory usage: 121.6 KB


In [126]:
train_X.head()

,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,is_holiday,year,month,day,hour,day_of_week,hour_sin,hour_cos,dow_sin,dow_cos,month_sin,month_cos
0,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,0,2012,10,2,9,1,7.071068e-01,-0.707107,0.781831,0.62349,-0.866025,0.5
1,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,0,2012,10,2,10,1,5.000000e-01,-0.866025,0.781831,0.62349,-0.866025,0.5
2,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,0,2012,10,2,11,1,2.588190e-01,-0.965926,0.781831,0.62349,-0.866025,0.5
3,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,0,2012,10,2,12,1,1.224647e-16,-1.000000,0.781831,0.62349,-0.866025,0.5
4,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,0,2012,10,2,13,1,-2.588190e-01,-0.965926,0.781831,0.62349,-0.866025,0.5


In [127]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   temp                 48204 non-null  float64       
 1   rain_1h              48204 non-null  float64       
 2   snow_1h              48204 non-null  float64       
 3   clouds_all           48204 non-null  int64         
 4   weather_main         48204 non-null  object        
 5   weather_description  48204 non-null  object        
 6   date_time            48204 non-null  datetime64[ns]
 7   is_holiday           48204 non-null  int64         
 8   year                 48204 non-null  int32         
 9   month                48204 non-null  int32         
 10  day                  48204 non-null  int32         
 11  hour                 48204 non-null  int32         
 12  day_of_week          48204 non-null  int32         
 13  hour_sin             48204 non-

In [128]:
X.loc[X["year"] == 2012, "month"].unique()

array([10, 11, 12], dtype=int32)

In [129]:
X['hour'].head(40)

0      9
1     10
2     11
3     12
4     13
5     14
6     15
7     16
8     17
9     18
10    19
11    20
12    21
13    22
14    23
15     0
16     1
17     2
18     3
19     4
20     5
21     6
22     8
23     9
24    12
25    13
26    14
27    15
28    16
29    18
30    19
31    20
32    21
33    22
34    23
35     0
36     1
37     2
38     3
39     4
Name: hour, dtype: int32

In [130]:
X.iloc[37,:]

temp                                283.17
rain_1h                                0.0
snow_1h                                0.0
clouds_all                               1
weather_main                         Clear
weather_description           sky is clear
date_time              2012-10-04 02:00:00
is_holiday                               0
year                                  2012
month                                   10
day                                      4
hour                                     2
day_of_week                              3
hour_sin                               0.5
hour_cos                          0.866025
dow_sin                           0.433884
dow_cos                          -0.900969
month_sin                        -0.866025
month_cos                              0.5
Name: 37, dtype: object

In [131]:
X.iloc[38,:]

temp                                282.04
rain_1h                                0.0
snow_1h                                0.0
clouds_all                               1
weather_main                         Clear
weather_description           sky is clear
date_time              2012-10-04 03:00:00
is_holiday                               0
year                                  2012
month                                   10
day                                      4
hour                                     3
day_of_week                              3
hour_sin                          0.707107
hour_cos                          0.707107
dow_sin                           0.433884
dow_cos                          -0.900969
month_sin                        -0.866025
month_cos                              0.5
Name: 38, dtype: object